In [1]:
import pickle
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import datetime as dt
import pandas as pd
from scipy import stats
from pyearth import Earth#conda install sklearn-contrib-py-earth
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

np.warnings.filterwarnings('ignore') #Runtime warning

In [2]:
#Header of csv: 'Date[0],Time[1],Depth of Airgun(m)[2],Depth of Reciever(m)[3],X Airgun[4],Y Airgun[5],Z Airgun[6],X_R1[7],Y_R1[8],Z_R1[9],SEL1[10],RMS1[11],SEL2[12],RMS2[13],SEL3[14],RMS3[15],SEL4[16],RMS4[17],SEL_full[18],RMS_full[19],T90_1[20],T90_2[21],T90_3[22],T90_4[23],T90_full[24]
def prepData(x_data,y_data,path):
    csv_train_path = path
    files = os.listdir(csv_train_path)
    dataTrain = {}
    x_data2 = x_data
    y_data2 = y_data
    for filename in sorted(files):
        dirname = csv_train_path + filename
        dataTrain = pd.read_csv(dirname)
        dataTrain = dataTrain.rename(columns={'Depth of Airgun(m)': 'ag_depth','Depth of Reciever(m)': 'hyd_depth'})
        dataTrain = dataTrain[(dataTrain.hyd_depth != -99999)]    
        ag_x = dataTrain['X Airgun']
        ag_y = dataTrain['Y Airgun']
        ag_z = dataTrain['Z Airgun']
        hyd_x = dataTrain['X_R1'] #relative to ag_x
        hyd_y = dataTrain['Y_R1'] #relative to ag_y
        hyd_z = dataTrain['Z_R1'] #relative to ag_z
        r_train = np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z))
        logr_train = np.log10(np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z)))
        dataTrain = dataTrain.assign(r_train = r_train,logr_train = logr_train,filename = filename)
        dataTrain = dataTrain.dropna(axis=0, how='any')
        
        #outliers
        sel_dif = {}
        sel_dif = dataTrain['SEL_full'].diff()
        sel_dif2 = sel_dif.dropna(axis=0, how='any')
        sel_thres = np.abs(np.mean(np.abs(sel_dif2)))
        sel_dif = sel_dif[np.abs(sel_dif.values) < sel_thres]
        dataTrain = dataTrain.loc[sel_dif.index]
        """
        rms_dif = {}
        rms_dif = dataTrain['RMS_full'].diff()
        rms_dif2 = rms_dif.dropna(axis=0, how='any')
        rms_thres = np.abs(np.mean(np.abs(rms_dif2)))
        rms_dif = rms_dif[np.abs(rms_dif.values) < rms_thres]
        dataTrain = dataTrain.loc[rms_dif.index]        
        """
        x_data2 = x_data2.append(dataTrain[['ag_depth', 'hyd_depth','r_train','logr_train']],ignore_index=True)
        y_data2 = y_data2.append(dataTrain[['SEL_full','RMS_full']], ignore_index=True)
        #data = data.append(dataTrain[['filename','ag_depth', 'hyd_depth','logr_train','r_train','RMS_full','SEL_full']],ignore_index=True)
    return [x_data2,y_data2]

def pickle2(filename,x_data,y_data):
    outfile = open(filename,'wb')
    pickle.dump({'X': x_data, 'SEL': y_data['SEL_full'], 'RMS': y_data['RMS_full']},outfile)
    outfile.close()
    return;

In [23]:
"""
#remove outliers and put data in frames
path5 = r'D:\Machine Learning\NewCSV\CSV1\Line_05_(RMS_and_SEL)\\'
path6 = r'D:\Machine Learning\NewCSV\CSV1\Line_06_(RMS_and_SEL)\\'
path7 = r'D:\Machine Learning\NewCSV\CSV1\Line_07_(RMS_and_SEL)\\'
[x_data,y_data] = prepData(pd.DataFrame(),pd.Series(),path5)
pickle2('line05.pickle',x_data,y_data)
[x_data,y_data] = prepData(x_data,y_data,path6)
[x_data,y_data] = prepData(x_data,y_data,path7)
pickle2('3lines.pickle',x_data,y_data)
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 70-71: malformed \N character escape (<ipython-input-23-a35b93605600>, line 11)

In [31]:
#draw from the pickle
infile = open('line05.pickle','rb')
#infile = open('3lines.pickle','rb')
data = pickle.load(infile, encoding='bytes')
x_train, x_test, y_train, y_test = train_test_split(data['X'],data['SEL'],test_size=0.1,shuffle=True)

In [32]:
#Linear regression
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
# Train the model using the training sets
linreg_model = reg.fit(x_train, y_train)
print('Coefficients (slope): \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

 
y_pred = linreg_model.predict(x_test)

R2 = reg.score(x_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)


Coefficients (slope): 
 [-4.99768287e-03  9.44219028e-04  5.96144836e-04 -9.65872405e+00]
Intercept: 
 200.0696844028013
R^2 score: 0.62 

Total Mean squared error: 8.81 

Subspace Mean squared error: 6.91 



In [33]:
#2nd degree polynomial
poly = PolynomialFeatures(degree=2)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Coefficients (slope): 
 [ 0.00000000e+00 -1.44837184e-02 -5.01492142e-03 -7.60742919e-01
 -7.15252040e+02 -1.04735513e-05  1.97942994e-05 -5.00895147e-07
  2.84631795e-03 -8.15781332e-06 -2.36097926e-08  1.03856977e-03
 -4.33228443e-06  1.83165500e-01  1.71371025e+02]
Intercept: 
 994.9284690879745
R^2 score: 0.65 

Total Mean squared error: 7.94 

Subspace Mean squared error: 6.26 



In [34]:
#3rd degree polynomial
poly = PolynomialFeatures(degree=3)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)
#print(y_pred.shape)
R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Coefficients (slope): 
 [ 1.87222485e-06 -1.57449655e+00  1.46756825e+00 -1.48281763e+02
 -7.14506059e+02 -1.18246139e-04  3.54538942e-04  1.39807609e-03
  1.26593589e+00 -2.28570515e-04 -1.28408592e-03 -1.18323775e+00
  3.22715609e-03  7.32276392e+01 -1.14332161e+03 -6.95338759e-08
  1.36203513e-07 -7.04385117e-09  7.04177775e-05 -4.79377555e-08
  1.62577258e-08 -2.01312410e-04  9.12617171e-09 -3.41648682e-04
 -2.84399488e-01 -2.45090924e-08 -1.13466169e-08  1.39577005e-04
 -8.32696490e-09  3.15922921e-04  2.58180403e-01  3.04567827e-09
 -6.41629041e-04 -9.71689714e+00  8.97444114e+02]
Intercept: 
 3146.731809587996
R^2 score: 0.68 

Total Mean squared error: 7.34 

Subspace Mean squared error: 6.94 



In [8]:
#this takes a while
"""
mars = Earth()
mars.fit(x_train,y_train)

filename = 'mars.pickle'
outfile = open(filename,'wb')

pickle.dump(mars,outfile)
outfile.close()
"""

"\nmars = Earth()\nmars.fit(x_train,y_train)\n\nfilename = 'mars.pickle'\noutfile = open(filename,'wb')\n\npickle.dump(mars,outfile)\noutfile.close()\n"

In [9]:
infile = open('mars.pickle','rb')
mars = pickle.load(infile)
y_pred = mars.predict(x_test)

R2 = mars.grsq_#MARS score equivalent
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = mars.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

R^2 score: 0.66 

Total Mean squared error: 55.45 

Subspace Mean squared error: 49.95 

